In [2]:
import os
from transformers import (
    LlamaForCausalLM,
    LlamaTokenizer,
)
from peft import PeftModel
import sys
import pandas as pd
import os
import torch
import re

In [13]:
code = "diabetes"
text_column = "Text"
label_column = "Text_label"
prompt = "prompt2"
data_path = "/scratch/aachary/Med-BERT/llama2/data"
output = "MIMIC_finetune_small_"
#output=""
# chkpt = "/checkpoint-30000"
chkpt="/checkpoint-2000"
save_name = output+"_"+code+"_"+prompt+'_yes_no_'+chkpt[1:]


In [14]:
ckpt  = "/scratch/aachary/Med-BERT/llama2/outputs/" +output+code+"_"+prompt+'_yes_no'+chkpt

In [ ]:
if code=="oud":
    code_text = "Opioid Use Disorder"
elif code == "sud":
    code_text = "Substance Use Disorder"
elif code == "diabetes":
    code_text = "Diabetes"
else:
    print("Error in the code")

In [16]:
data_path

'/scratch/aachary/Med-BERT/llama2/data'

In [17]:
max_length = 4096

In [18]:
model_id = '/scratch/aachary/Med-BERT/llama2/Llama-2-7b-chat-hf'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [19]:
test_data = pd.read_csv(os.path.join("/scratch/aachary/Med-BERT/llama2/data/",output+"test_"+code+"_"+prompt+ ".csv"))

In [22]:
def create_test_prompt(examples):
    # Initialize static strings for the prompt template
\    INTRO_BLURB = "Given a patient's past medical history, predict whether the patient will have a future diagnosis of " + code_text + ". Return 'Yes' or 'No' after the XML tag <Diagnosis>."
    INSTRUCTION_KEY = "### Instruction:"
    INPUT_KEY = "### Input:"
    RESPONSE_KEY = "### Response:"
    END_KEY = "### End"
    
    instruction = f"{INSTRUCTION_KEY}\n{INTRO_BLURB}"
    input_context = f"{INPUT_KEY}\n{examples[text_column]}" if examples[text_column] else None

    high_low_label = examples[label_column]
    if high_low_label == "High":
        t_label = "Yes"
    elif high_low_label == "Low":
        t_label = "No"
    else:
        print("There is some error with the label")
        
    response_ground_truth = f"{RESPONSE_KEY}\n<Diagnosis>"
    
    end = f"{END_KEY}"

    # Create a list of prompt template elements
    parts_ground_truth = [part for part in [instruction, input_context, response_ground_truth] if part]
    # Join prompt template elements into a single string to create the prompt template
    formatted_prompt_ground_truth = "\n\n".join(parts_ground_truth)

    # # Store the formatted prompt template in a new key "text"
    # examples["prompt"] = formatted_prompt

    return formatted_prompt_ground_truth


In [23]:
org_model =  LlamaForCausalLM.from_pretrained(
    model_id, 
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [24]:
test_model = PeftModel.from_pretrained(
    org_model, ckpt
)

In [25]:
test_model.to(device)


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear(
                in_features=4096, out_features=4096, bias=False
                (lora_dr

In [26]:
test_model = test_model.merge_and_unload()

In [27]:
list_examples =[]
for i,row in test_data.iterrows():
    e = create_test_prompt(row)
    list_examples.append(e)


In [58]:
tokenizer = LlamaTokenizer.from_pretrained(model_id)

In [39]:
import numpy as np

In [40]:
labels = list(test_data["Label"].values)

In [63]:
pred = []
import time
import datetime

for i,example in enumerate(list_examples):
    print(i)
    model_input = tokenizer(example, return_tensors="pt").to("cuda")
    # print(f'Tokenize: {datetime.timedelta(seconds=time.time()-start)}')
    # start = time.time()
    
    output_ = test_model(**model_input)
    next_token_logits = output_.logits[0, -1, :]
   
    # 2. step to convert the logits to probabilities
    next_token_probs = torch.softmax(next_token_logits, -1)
    
    # 3. step to get the top 20
    topk_next_tokens= torch.topk(next_token_probs, 20)
    
    low_tokens = {"No", "No", "N", "no", "NO"}
    high_tokens = {"Yes", "Yes", "yes", "yes", "YES", "Y"}

    #putting it together  
    top_k_probs = [(tokenizer.decode(idx), prob) for idx, prob in zip(topk_next_tokens.indices, topk_next_tokens.values)]
    low_sum = 0
    high_sum = 0
    
    for k, v in top_k_probs:
        if k in low_tokens:
            low_sum+=v.item()
        elif k in high_tokens:
            high_sum+=v.item()
    arr = [high_sum, low_sum]
    low_high_probs = np.exp(arr) / np.sum(np.exp(arr), axis=0)
    # print("Predicted Probability", low_high_probs)
    # print("Label",labels[i]) 
    # if labels[i]==1:
    #     print(low_high_probs[0])
    #     print(top_k_probs)
    pred.append(low_high_probs[0])
    # print(f'Sum: {datetime.timedelta(seconds=time.time()-start)}')
    # start = time.time()
    
    

0
[0.04241981357336044, 0.953985794345499]
1
[0.9999942739111702, 2.7069579573435476e-06]
2
[0.036473539636403984, 0.9623989219622899]
3
[0.008478897093027626, 0.9912803285893688]
4
[0.4237843022656307, 0.5746395214664517]
5
[0.0033820249870473162, 0.9965980413908255]
6
[0.2796197100124118, 0.7198708651628749]
7
[0.10858881347667193, 0.8895546509320411]
8
[0.03524043170955338, 0.9640042976634504]
9
[0.0277638743651778, 0.9720605513384726]
10
[0.18161086966938456, 0.8161333703901619]
11
[0.008198265701878427, 0.9917073367922171]
12
[0.07062497747074303, 0.9288060973940446]
13
[0.999999562180399, 3.8154595927153423e-07]
14
[0.08516985923051834, 0.898311095777899]
15
[0.01346546482204758, 0.9863061731075504]
16
[0.08016159385442734, 0.8857317075598985]
17
[0.21362654067888798, 0.784276494756341]
18
[0.2808792715331947, 0.7184635231060383]
19
[0.1375018855687813, 0.8615829857380959]
20
[0.009398426108191416, 0.9903596457847925]
21
[0.056882781245803926, 0.9426221946291662]
22
[0.0224850795

In [65]:
import pickle
from sklearn.metrics import roc_auc_score, average_precision_score

In [66]:
df = pd.DataFrame(columns =["PatientID","Predictions","Label"])

In [67]:
df["PatientID"] = test_data["PatientId"].values
df["Predictions"] = pred
df["Label"] = labels

In [68]:
df

,PatientID,Predictions,Label
0,10002428,0.286679,0
1,10021179,0.731057,1
2,10023349,0.283752,0
3,10026011,0.272336,0
4,10071129,0.462358,0
5,10095024,0.270277,0
6,10100170,0.391681,0
7,10110724,0.314112,0
8,10126529,0.283176,0
9,10138401,0.280033,0


In [69]:
df.to_csv("Predictions_" +save_name+".csv", index=False)

In [70]:
roc = roc_auc_score(labels, pred)


In [71]:
roc

0.8504273504273504

In [72]:
pr_auc = average_precision_score(labels, pred)

In [73]:
pr_auc

0.7507805989779426

In [ ]:
acc = accuracy_score(labels, [1 if i>=0.5 else 0 for i in pred])

In [ ]:
acc